In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime


In [33]:
signal_1 = {
    'name':'A',
    'nu_time':'2022-10-04T15:19:06.319905',}
signal_2 = {
    'name':'B',
    'nu_time':'2022-10-04T15:19:08.319905',}
signal_3 = {
    'name':'C',
    'nu_time':'2022-10-04T15:19:15.319905',}
signal_4 = {
    'name':'D',
    'nu_time':'2022-10-04T15:19:18.319905'}
signal_5 = {
    'name':'E',
    'nu_time':'2022-10-04T15:19:26.319905'}
signal_6 = {
    'name':'F',
    'nu_time':'2022-10-04T15:19:30.319905'
}
signal_7  = {
     'name':'C',
    'nu_time':'2022-10-04T15:19:19.319905'
}

In [19]:
class CoincidenceDataHanddler:
    def __init__(self):
        self.df = pd. DataFrame()
        self.current_cache_state = {}

    def add_to_cahce(self, message):
        # consider this the 'main' method this class
        self._garbage_checker()
        message['nu_time_as_datetime'] = datetime.fromisoformat(message['nu_time'])
        self._manage_cache(message)
        self.df = self.df.sort_values(by=['sub_group','nu_time_delta'],ignore_index=True)
        self.df = self.df.reset_index(drop=True)


    def _manage_cache(self,message):
        if len(self.df) == 0:
            print('Initial Message!!')
            message['nu_time_delta'] = 0
            message['sub_group'] = 0
            temp = pd.DataFrame([message])
            self.df = pd.concat([self.df,temp], ignore_index=True)
            # self.current_cache_state = {1:1}
        else:
            self._check_coinc_in_subgroups(message)

    def _check_coinc_in_subgroups(self, message):
        sub_group_tags = self.df['sub_group'].unique()
        coinc = True
        for tag in sub_group_tags:
            sub_df = self.df.query('sub_group==@tag')
            sub_df = sub_df.reset_index(drop=True)
            sub_ini_t = sub_df['nu_time_as_datetime'][0]
            delta = (message['nu_time_as_datetime'] - sub_ini_t).total_seconds()
            if 0 < delta  <= 10.0:
                message['sub_group'] = tag
                message['nu_time_delta'] = delta
                temp = pd.DataFrame([message])
                self.df = pd.concat([self.df,temp], ignore_index=True)

                coinc = True
            else:
                coinc = False
                continue
        if not coinc:
            new_ini_t = message['nu_time_as_datetime']
            new_sub_tag = len(sub_group_tags)
            message_as_df = pd.DataFrame([message])
            temp_cache = pd.concat([self.df,message_as_df], ignore_index=True)
            temp_cache = temp_cache.drop_duplicates(subset=['name', 'nu_time'])
            print(temp_cache)
            temp_cache['nu_time_delta'] = (temp_cache['nu_time_as_datetime']- new_ini_t).dt.total_seconds()
            # Make two subgroup one for early signal and post
            new_sub_group_early = temp_cache.query('-10 <= nu_time_delta <= 0')
            new_sub_group_post = temp_cache.query('0 <= nu_time_delta <= 10.0')
            new_sub_group_early['sub_group'] = new_sub_tag
            new_sub_group_post['sub_group'] = new_sub_tag + 1

            new_sub_group_early = new_sub_group_early.sort_values(by='nu_time_as_datetime')
            new_sub_group_post = new_sub_group_post.sort_values(by='nu_time_as_datetime')

            self._organize_cache(sub_group=new_sub_group_post)
            self._organize_cache(sub_group=new_sub_group_early)

    def _check_for_redundancies(self,sub_group):
        names = sub_group['name']
        t = sub_group['nu_time']
        if len(sub_group) == 1:
            return True
        for sub_tag in self.df['sub_group'].unique():
            other_sub = self.df.query('sub_group == @sub_tag')
            check_t = names.isin(other_sub['name'])
            check_name = names.isin(other_sub['nu_time'])

            all_t_redundant = check_t.eq(True).all()
            all_names_redundant = check_name.eq(True).all()

            if all_names_redundant and all_t_redundant:
                return True

        return False

    def _organize_cache(self, sub_group):
        if self._check_for_redundancies(sub_group):
            return
        sub_group = sub_group.reset_index(drop=True)
        if sub_group['nu_time_delta'][0] < 0:
            sub_group['nu_time_delta'] = sub_group['nu_time_delta'] - sub_group['nu_time_delta'][0]

        self.df = pd.concat([self.df,sub_group], ignore_index=True)
        self.df = self.df.sort_values(by=['sub_group','nu_time_as_datetime']).reset_index(drop=True)



    def _garbage_checker(self, ):
        pass

    def cache_retraction(self, retraction_message):
        retracted_name = retraction_message['name']
        self.df = self.df.query('name!=@retracted_name')



In [20]:
coinc = CoincidenceDataHanddler()

In [21]:
coinc.add_to_cahce(signal_1)

Initial Message!!


In [22]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0,0


In [23]:
coinc.add_to_cahce(signal_3)
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0


In [24]:
coinc.add_to_cahce(signal_2)

In [25]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,2.0,0
2,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0


In [26]:
coinc.add_to_cahce(signal_4)

  name                     nu_time        nu_time_as_datetime  nu_time_delta  \
0    A  2022-10-04T15:19:06.319905 2022-10-04 15:19:06.319905            0.0   
1    B  2022-10-04T15:19:08.319905 2022-10-04 15:19:08.319905            2.0   
2    C  2022-10-04T15:19:15.319905 2022-10-04 15:19:15.319905            9.0   
3    D  2022-10-04T15:19:18.319905 2022-10-04 15:19:18.319905            NaN   

   sub_group  
0        0.0  
1        0.0  
2        0.0  
3        NaN  


/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_early['sub_group'] = new_sub_tag
/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_post['sub_group'] = new_sub_tag + 1


In [27]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,2.0,0
2,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0
3,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,0.0,1
4,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,7.0,1
5,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,10.0,1


In [28]:
coinc.add_to_cahce(signal_5)

  name                     nu_time        nu_time_as_datetime  nu_time_delta  \
0    A  2022-10-04T15:19:06.319905 2022-10-04 15:19:06.319905            0.0   
1    B  2022-10-04T15:19:08.319905 2022-10-04 15:19:08.319905            2.0   
2    C  2022-10-04T15:19:15.319905 2022-10-04 15:19:15.319905            9.0   
5    D  2022-10-04T15:19:18.319905 2022-10-04 15:19:18.319905           10.0   
6    E  2022-10-04T15:19:26.319905 2022-10-04 15:19:26.319905            NaN   

   sub_group  
0        0.0  
1        0.0  
2        0.0  
5        1.0  
6        NaN  


/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_early['sub_group'] = new_sub_tag
/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_post['sub_group'] = new_sub_tag + 1


In [29]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,2.0,0
2,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0
3,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,0.0,1
4,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,7.0,1
5,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,10.0,1
6,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,0.0,2
7,E,2022-10-04T15:19:26.319905,2022-10-04 15:19:26.319905,8.0,2


In [30]:
coinc.add_to_cahce(signal_6)

  name                     nu_time        nu_time_as_datetime  nu_time_delta  \
0    A  2022-10-04T15:19:06.319905 2022-10-04 15:19:06.319905            0.0   
1    B  2022-10-04T15:19:08.319905 2022-10-04 15:19:08.319905            2.0   
2    C  2022-10-04T15:19:15.319905 2022-10-04 15:19:15.319905            9.0   
5    D  2022-10-04T15:19:18.319905 2022-10-04 15:19:18.319905           10.0   
7    E  2022-10-04T15:19:26.319905 2022-10-04 15:19:26.319905            8.0   
8    F  2022-10-04T15:19:30.319905 2022-10-04 15:19:30.319905            NaN   

   sub_group  
0        0.0  
1        0.0  
2        0.0  
5        1.0  
7        2.0  
8        NaN  


/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_early['sub_group'] = new_sub_tag
/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_post['sub_group'] = new_sub_tag + 1


In [31]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,2.0,0
2,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0
3,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,0.0,1
4,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,7.0,1
5,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,10.0,1
6,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,0.0,2
7,E,2022-10-04T15:19:26.319905,2022-10-04 15:19:26.319905,8.0,2
8,E,2022-10-04T15:19:26.319905,2022-10-04 15:19:26.319905,0.0,3
9,F,2022-10-04T15:19:30.319905,2022-10-04 15:19:30.319905,4.0,3


In [34]:
coinc.add_to_cahce(signal_7)

   name                     nu_time        nu_time_as_datetime  nu_time_delta  \
0     A  2022-10-04T15:19:06.319905 2022-10-04 15:19:06.319905            0.0   
1     B  2022-10-04T15:19:08.319905 2022-10-04 15:19:08.319905            2.0   
2     C  2022-10-04T15:19:15.319905 2022-10-04 15:19:15.319905            9.0   
5     D  2022-10-04T15:19:18.319905 2022-10-04 15:19:18.319905           10.0   
7     E  2022-10-04T15:19:26.319905 2022-10-04 15:19:26.319905            8.0   
9     F  2022-10-04T15:19:30.319905 2022-10-04 15:19:30.319905            4.0   
10    C  2022-10-04T15:19:19.319905 2022-10-04 15:19:19.319905            1.0   

    sub_group  
0           0  
1           0  
2           0  
5           1  
7           2  
9           3  
10          2  


/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_early['sub_group'] = new_sub_tag
/var/folders/mr/0p5djr157hxdh7bb2gmzywyr0000gn/T/ipykernel_62599/3618793183.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_sub_group_post['sub_group'] = new_sub_tag + 1


In [35]:
coinc.df

,name,nu_time,nu_time_as_datetime,nu_time_delta,sub_group
0,A,2022-10-04T15:19:06.319905,2022-10-04 15:19:06.319905,0.0,0
1,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,2.0,0
2,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,9.0,0
3,B,2022-10-04T15:19:08.319905,2022-10-04 15:19:08.319905,0.0,1
4,C,2022-10-04T15:19:15.319905,2022-10-04 15:19:15.319905,7.0,1
5,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,10.0,1
6,D,2022-10-04T15:19:18.319905,2022-10-04 15:19:18.319905,0.0,2
7,C,2022-10-04T15:19:19.319905,2022-10-04 15:19:19.319905,1.0,2
8,E,2022-10-04T15:19:26.319905,2022-10-04 15:19:26.319905,8.0,2
9,E,2022-10-04T15:19:26.319905,2022-10-04 15:19:26.319905,0.0,3
